# Remapping: Analysis of pairs of neurons during open field foraging, light trials and dark trials

We get the IFR and the firing rate maps for the neurons in different conditions. We then look at the IFR association and firing rate map similarity for pairs of neurons in a given condition.

* Do cells that fire together in one environment also fire together in the next environment?
* Do cells with similar firing rate maps in one condition also have similar maps in another?


In [1]:
%load_ext autoreload
%autoreload 2
%run setup_project.py
prepareSessionsForSpatialAnalysisProject(sSesList,myProject.sessionList)

Project name: autopi_ca1
dataPath: /adata/projects/autopi_ca1
dlcModelPath: /adata/models
Reading /adata/projects/autopi_ca1/sessionList
We have 39 testing sessions in the list
See myProject and sSesList objects
Loading Animal_pose and Spike_train, sSes.ap and sSes.cg


100%|███████████████████████████████████████████| 39/39 [00:17<00:00,  2.20it/s]

Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


In [2]:
for ses, sSes in tqdm(zip(myProject.sessionList,sSesList)):
    getSearchHomingIntervals(ses,sSes)

39it [00:00, 100.69it/s]


## Instantaneous Firing rate associations

We can look at pairs of cells and test if their firing associations (r values) are preserve or change across different conditions.
The function below calculates the firing associations of pairs of neurons in different conditions.
Later on we can compare IFR association across conditions.

In [3]:
from scipy.stats import pearsonr
def ifrAssociationsPerCondition(ses,sSes,conditions=["circ80","circ80_1","circ80_2","all_task","all_task_1","all_task_2","light","dark","light_1","light_2","dark_1","dark_2","all_light","all_dark","all_light_1","all_light_2","all_dark_1","all_dark_2"],prefix="ifrAsso_"):
    """
    Calculate the instantaneous firing rate associations of pairs of neurons in a sets of conditions
    
    Arguments: 
    ses: autopipy session
    sSes: spikeA session
    conditions: list of condition names you want to analyze
    prefix: prefix for the DataFrame column names
    
    Return:
    Pandas DataFrame with the IFR association for each condition. One row per pair
    """ 
    # we will store the results in a dictionary, one list of scores per condition
    res={}
    for cond in conditions:
        # set the intervals for each neuron and calculate the IFR
        for n in sSes.cg.neuron_list:
            n.spike_train.unset_intervals()
            n.spike_train.set_intervals(ses.intervalDict[cond])
            n.spike_train.instantaneous_firing_rate(bin_size_sec = 0.100, sigma = 1,outside_interval_solution="remove")
        # calculate firing association for all pairs of neurons
        sSes.cg.make_pairs(pair_type="combinations")
        rs = np.empty(len(sSes.cg.pairs)) # to store the results
        for i, (j,k) in enumerate(sSes.cg.pairs):
            n1=sSes.cg.neuron_list[j]
            n2=sSes.cg.neuron_list[k]
            rs[i] = pearsonr(n1.spike_train.ifr[0].flatten(),n2.spike_train.ifr[0].flatten())[0]
        res[cond]=rs
            
    #create a DataFrame from the dictionary
    res = pd.DataFrame(res)
    res = res.add_prefix(prefix)
    res["id1"] = [ sSes.name+"_"+sSes.cg.neuron_list[i].name for i,j in sSes.cg.pairs]
    res["id2"] = [ sSes.name+"_"+sSes.cg.neuron_list[j].name for i,j in sSes.cg.pairs]
    
    res["session"] = sSes.name
    return res

def spikeTimeCrosscorrelation(ses,sSes):
    """
    To make sure the pairs do not have common refractory periods
    """
    for n in sSes.cg.neuron_list:
        n.spike_train.unset_intervals()
        
    sSes.cg.make_pairs(pair_type="combinations")
    
    # get the size of one crosscorrelation
    st0 = sSes.cg.neuron_list[0].spike_train
    st1 = sSes.cg.neuron_list[1].spike_train
    myHist,myRange = st0.spike_time_crosscorrelation(st1=None,st2=st1,bin_size_sec=0.0005, min_sec=-0.05, max_sec=0.05)
    res = np.empty((len(sSes.cg.pairs),myHist.shape[0]))
    for i, (j,k) in enumerate(sSes.cg.pairs):
        n1=sSes.cg.neuron_list[j]
        n2=sSes.cg.neuron_list[k]
        myHist,myRange = n1.spike_train.spike_time_crosscorrelation(st1=None,st2=n2.spike_train, bin_size_sec=0.0005, min_sec=-0.05, max_sec=0.05)
        res[i,:]=myHist
    return res


Run on one session

In [4]:
ses.intervalDict.keys()

dict_keys(['circ80', 'circ80_1', 'circ80_2', 'task', 'task_1', 'task_2', 'light', 'light_1', 'light_2', 'dark', 'dark_1', 'dark_2', 'searchPath_light', 'searchPath_light_1', 'searchPath_light_2', 'searchPath_dark', 'searchPath_dark_1', 'searchPath_dark_2', 'searchToLeverPath_light', 'searchToLeverPath_light_1', 'searchToLeverPath_light_2', 'searchToLeverPath_dark', 'searchToLeverPath_dark_1', 'searchToLeverPath_dark_2', 'homingPath_light', 'homingPath_light_1', 'homingPath_light_2', 'homingPath_dark', 'homingPath_dark_1', 'homingPath_dark_2', 'all_light', 'all_light_1', 'all_light_2', 'all_dark', 'all_dark_1', 'all_dark_2', 'all_light_noPress', 'all_dark_noPress', 'all_task', 'all_task_1', 'all_task_2'])

In [5]:
%%time
ses,sSes = list(zip(myProject.sessionList,sSesList))[0]
res = ifrAssociationsPerCondition(ses,sSes)
res = spikeTimeCrosscorrelation(ses,sSes)

/home/kevin/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


CPU times: user 5.42 s, sys: 3.15 s, total: 8.56 s
Wall time: 5.05 s


In [6]:
len(myProject.sessionList)

39

Run on all sessions

In [7]:
res_ifr=pd.concat([ifrAssociationsPerCondition(ses,sSes) for ses,sSes in tqdm(zip(myProject.sessionList,sSesList))],ignore_index=True)

39it [10:38, 16.37s/it]


In [8]:
res_cc=np.concatenate([spikeTimeCrosscorrelation(ses,sSes) for ses,sSes in tqdm(zip(myProject.sessionList,sSesList))])

39it [00:31,  1.23it/s]


## Firing rate map similarities

As a complementary analysis to the IFR association, we can also look at firing rate map similarity in different conditions. 


In [9]:
from scipy.stats import pearsonr
def map_cor(a,b):
    """
    Correlation coefficient between two firing rate maps
    
    Arguments:
    a: 2D np.array (map1)
    b: 2D np.array (map2)
    
    Returns:
    Pearson correlation coefficient between a and b
    """
    a = a.flatten()
    b = b.flatten()
    indices = np.logical_and(~np.isnan(a), ~np.isnan(b))
    r,p = pearsonr(a[indices],b[indices])
    return r

def mapSimilarityPerCondition(ses,sSes,
                              conditions=["circ80","circ80_1","circ80_2","all_task","all_task_1","all_task_2","light","dark","light_1","light_2","dark_1","dark_2","all_light","all_dark","all_light_1","all_light_2","all_dark_1","all_dark_2"],
                                          prefix="mapSim_"):
    """
    Calculate the map similarity of pairs of neurons in a sets of conditions
    
    Arguments: 
    ses: autopipy session
    sSes: spikeA session
    conditions: list of condition names you want to analyze
    prefix: prefix for the DataFrame column names
    
    Return:
    Pandas DataFrame with map similarity for each condition. One row per pair
    """ 
    # we will store the results in a dictionary, one list of scores per condition
    res={}
    xy_range=np.array([[-50,-90],[50,60]])
    
    for cond in conditions:
        # set the intervals for each neuron and animal pose and calculate the maps
        
        sSes.ap.set_intervals(ses.intervalDict[cond])
        
        for n in sSes.cg.neuron_list:
            n.spike_train.unset_intervals()
            n.spike_train.set_intervals(ses.intervalDict[cond])
            n.spatial_properties.firing_rate_map_2d(cm_per_bin =3, smoothing_sigma_cm = 5, smoothing=True,xy_range=xy_range)
            
            
        # calculate map similarity for all pairs of neurons
        sSes.cg.make_pairs(pair_type="combinations")
        rs = np.empty(len(sSes.cg.pairs)) # to store the results
        for i, (j,k) in enumerate(sSes.cg.pairs):
            n1=sSes.cg.neuron_list[j]
            n2=sSes.cg.neuron_list[k]
            rs[i] = map_cor(n1.spatial_properties.firing_rate_map,n2.spatial_properties.firing_rate_map)
        res[cond]=rs
            
    #create a DataFrame from the dictionary
    res = pd.DataFrame(res)
    res = res.add_prefix(prefix)
    res["id1"] = [ sSes.name+"_"+sSes.cg.neuron_list[i].name for i,j in sSes.cg.pairs]
    res["id2"] = [ sSes.name+"_"+sSes.cg.neuron_list[j].name for i,j in sSes.cg.pairs]
    
    res["session"] = sSes.name
    return res

In [10]:
ses,sSes = list(zip(myProject.sessionList,sSesList))[0]
res = mapSimilarityPerCondition(ses,sSes)
res

,mapSim_circ80,mapSim_circ80_1,mapSim_circ80_2,mapSim_all_task,mapSim_all_task_1,mapSim_all_task_2,mapSim_light,mapSim_dark,mapSim_light_1,mapSim_light_2,...,mapSim_dark_2,mapSim_all_light,mapSim_all_dark,mapSim_all_light_1,mapSim_all_light_2,mapSim_all_dark_1,mapSim_all_dark_2,id1,id2,session
0,0.188970,0.377550,-0.063779,0.205488,0.324524,0.182534,0.298867,0.340701,0.389660,0.118868,...,0.290365,0.273858,0.375962,0.339146,0.159948,0.213924,0.249719,mn5824-20112020-0107_80,mn5824-20112020-0107_90,mn5824-20112020-0107
1,0.397997,0.078335,0.539016,0.412078,0.098792,0.219923,0.015716,-0.260255,-0.042240,-0.007342,...,-0.135163,0.017873,-0.023686,-0.031132,0.154945,-0.054104,0.087172,mn5824-20112020-0107_80,mn5824-20112020-0107_92,mn5824-20112020-0107
2,-0.331926,NaN,-0.326266,-0.057822,-0.077159,-0.149833,-0.149790,0.030228,-0.162309,-0.229980,...,-0.099290,-0.443613,0.385690,-0.457705,-0.195193,0.224158,0.078136,mn5824-20112020-0107_80,mn5824-20112020-0107_96,mn5824-20112020-0107
3,-0.372722,-0.121031,-0.524874,0.034142,0.148018,-0.000975,0.045847,0.404098,0.119261,-0.324990,...,0.360234,-0.161663,0.155711,-0.290115,0.042610,-0.087486,0.276957,mn5824-20112020-0107_80,mn5824-20112020-0107_98,mn5824-20112020-0107
4,-0.114121,0.195360,-0.348479,-0.188524,-0.363811,-0.228677,-0.062324,-0.132594,-0.284955,0.301873,...,-0.020904,0.196144,-0.222498,0.155528,-0.198420,-0.305427,0.049718,mn5824-20112020-0107_80,mn5824-20112020-0107_100,mn5824-20112020-0107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,-0.485498,-0.433770,-0.474888,0.048948,0.058826,0.143506,0.383050,0.135416,0.170573,0.158104,...,0.130327,0.059608,0.114261,-0.147006,0.320317,0.116995,0.067372,mn5824-20112020-0107_116,mn5824-20112020-0107_122,mn5824-20112020-0107
62,NaN,NaN,NaN,-0.168673,-0.178337,-0.067167,-0.007998,-0.386727,-0.165835,0.168308,...,-0.361977,0.221828,-0.213034,0.243027,-0.348196,-0.173638,-0.214634,mn5824-20112020-0107_116,mn5824-20112020-0107_124,mn5824-20112020-0107
63,0.488112,0.603755,0.232816,-0.190029,-0.098970,-0.226168,0.555680,0.252100,0.196636,0.205709,...,0.343952,-0.131586,-0.125742,-0.121268,-0.186287,-0.006696,-0.180500,mn5824-20112020-0107_118,mn5824-20112020-0107_122,mn5824-20112020-0107
64,NaN,NaN,NaN,-0.103106,-0.112037,0.055867,-0.070380,-0.397552,-0.116966,0.269906,...,-0.267650,0.222053,-0.107162,0.048581,0.030576,-0.251609,0.025467,mn5824-20112020-0107_118,mn5824-20112020-0107_124,mn5824-20112020-0107


In [11]:
res_map=pd.concat([mapSimilarityPerCondition(ses,sSes) for ses,sSes in tqdm(zip(myProject.sessionList,sSesList))],ignore_index=True)

0it [00:00, ?it/s]/home/kevin/anaconda3/envs/DEEPLABCUT/lib/python3.8/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
39it [11:02, 16.99s/it]


## Check data integrity of ifr and mapSim df and merge

In [12]:
print(res_map.shape,res_ifr.shape)
if res_map.shape != res_ifr.shape:
    print("problem with the shape of the results")
else:
    print("shape of 2 dfs is matching")
    
if res_map.shape[0] != res_cc.shape[0]:
    print("problem with res_map and res_cc")

(29841, 21) (29841, 21)
shape of 2 dfs is matching


In [13]:
res_map.iloc[:,:-3]

,mapSim_circ80,mapSim_circ80_1,mapSim_circ80_2,mapSim_all_task,mapSim_all_task_1,mapSim_all_task_2,mapSim_light,mapSim_dark,mapSim_light_1,mapSim_light_2,mapSim_dark_1,mapSim_dark_2,mapSim_all_light,mapSim_all_dark,mapSim_all_light_1,mapSim_all_light_2,mapSim_all_dark_1,mapSim_all_dark_2
0,0.188970,0.377550,-0.063779,0.205488,0.324524,0.182534,0.298867,0.340701,0.389660,0.118868,0.232671,0.290365,0.273858,0.375962,0.339146,0.159948,0.213924,0.249719
1,0.397997,0.078335,0.539016,0.412078,0.098792,0.219923,0.015716,-0.260255,-0.042240,-0.007342,-0.239799,-0.135163,0.017873,-0.023686,-0.031132,0.154945,-0.054104,0.087172
2,-0.331926,NaN,-0.326266,-0.057822,-0.077159,-0.149833,-0.149790,0.030228,-0.162309,-0.229980,0.177510,-0.099290,-0.443613,0.385690,-0.457705,-0.195193,0.224158,0.078136
3,-0.372722,-0.121031,-0.524874,0.034142,0.148018,-0.000975,0.045847,0.404098,0.119261,-0.324990,0.147876,0.360234,-0.161663,0.155711,-0.290115,0.042610,-0.087486,0.276957
4,-0.114121,0.195360,-0.348479,-0.188524,-0.363811,-0.228677,-0.062324,-0.132594,-0.284955,0.301873,-0.293686,-0.020904,0.196144,-0.222498,0.155528,-0.198420,-0.305427,0.049718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29836,0.105399,0.190576,0.000304,-0.388058,-0.340658,-0.338184,-0.083830,-0.087357,-0.242720,-0.050973,-0.191722,0.012126,-0.226942,-0.280463,-0.194745,-0.230241,-0.321705,-0.228959
29837,0.020007,0.051041,0.009694,0.572778,0.734528,0.507359,0.149169,0.825215,0.143824,0.016221,0.812748,0.730757,0.107511,0.798212,0.142984,0.052827,0.759736,0.774551
29838,0.051649,0.108651,-0.078257,0.463088,0.472347,0.375941,0.078978,0.196718,0.119894,0.024408,0.372050,0.147696,0.298291,0.049394,0.092237,0.498627,-0.142262,0.424810
29839,0.171690,0.156684,0.173986,-0.438420,-0.369041,-0.447162,-0.086240,-0.081593,-0.031910,-0.212180,-0.028646,-0.112893,0.284551,-0.404726,0.093996,0.367020,-0.266139,-0.270946


# Save the data

In [14]:
res = pd.concat([res_ifr,res_map.iloc[:,0:-3]],axis=1)
fn=myProject.dataPath+"/results/pairs_ifrAsso_mapSim.csv"
print("Saving to",fn)
res.to_csv(fn,index=False)

Saving to /adata/projects/autopi_ca1/results/pairs_ifrAsso_mapSim.csv


In [15]:
fn=myProject.dataPath+"/results/pairs_cc.npy"
np.save(fn,res_cc)